# Visualize DLC results 

In [1]:
from ipyfilechooser import FileChooser
import os

try:
    %store -r filename
    filename = os.path.dirname(filename)
except NameError:
    filename = "//10.69.168.1/crnldata/forgetting/Aurelie/CheeseboardExperiment/DAQ_data/AB/"

fc1 = FileChooser(filename, select_default=True, show_only_dirs=False, title="<b>Choose .h5 file</b>")
display(fc1)

def update_file(chooser):
    global filename
    if chooser.selected:
        filename = chooser.selected
        %store filename

fc1.register_callback(update_file)

FileChooser(path='\\10.69.168.1\crnldata\forgetting\Carla\Cheeseboard\AHAD01.37\7-8 semaines\Habituation', fil…

Stored 'filename' (str)
Stored 'filename' (str)
Stored 'filename' (str)
Stored 'filename' (str)
Stored 'filename' (str)


In [21]:
df = pd.read_hdf(filename)

In [24]:
df

scorer    DLC_Resnet50_CheeseboardFeb6shuffle1_snapshot_160              \
bodyparts                                              Head               
coords                                                    x           y   
0                                                       NaN         NaN   
1                                                       NaN         NaN   
2                                                       NaN         NaN   
3                                                       NaN         NaN   
4                                                       NaN         NaN   
...                                                     ...         ...   
1370                                             956.196228  755.621521   
1371                                             956.279602  755.620361   
1372                                             956.178772  755.947815   
1373                                             956.076721  756.046387   
1374                                             956.170349  755.975037   

scorer                                                                \
bodyparts                   Back                           bodypart3   
coords    likelihood           x           y likelihood            x   
0           0.021467  691.795959  405.644287   0.007199   691.880737   
1           0.018284  691.852051  405.373901   0.004216   691.904907   
2           0.021266  691.784729  405.554230   0.006906   795.933167   
3           0.017296  691.860107  405.452118   0.004907   691.897461   
4           0.020090  691.843018  405.586853   0.006138   691.893677   
...              ...         ...         ...        ...          ...   
1370        0.987604  987.448242  753.042358   0.978459  1012.172791   
1371        0.989989  987.489563  752.907104   0.973659  1012.177734   
1372        0.989488  987.561218  752.966736   0.972038  1012.178772   
1373        0.989486  987.451172  752.873596   0.976297  1012.177429   
1374        0.985459  987.486633  752.936279   0.973719  1012.177246   

scorer                                                               
bodyparts                            objectA                         
coords              y likelihood           x           y likelihood  
0          435.977539   0.001059  739.962891  419.943329   0.000958  
1          435.976135   0.000916  739.964783  419.946106   0.000921  
2          372.004486   0.000815  739.964783  419.942780   0.000868  
3          435.976685   0.000922  739.967102  419.946045   0.000914  
4          435.977936   0.000853  739.962219  419.943176   0.000888  
...               ...        ...         ...         ...        ...  
1370       747.928894   0.004455  948.117310  763.987244   0.002249  
1371       747.933838   0.004468  948.121216  763.985046   0.002297  
1372       747.932190   0.004525  948.112366  763.991699   0.002258  
1373       747.932434   0.004481  948.109924  763.992004   0.002182  
1374       747.933960   0.004483  948.111633  763.991028   0.002265  

[1375 rows x 12 columns]

In [23]:
df.iloc[:, 0] = df.apply(lambda row: row.iloc[0] if row.iloc[2] > 0.9 else np.nan, axis=1)
df.iloc[:, 1] = df.apply(lambda row: row.iloc[1] if row.iloc[2] > 0.9 else np.nan, axis=1)

X = df.iloc[:, 0]
Y = df.iloc[:, 1]

In [38]:
606/120

5.05

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
import math
import os
from pathlib import Path
import re
%matplotlib widget

# Load HDF5 file
df = pd.read_hdf(filename)
directory = os.path.dirname(filename)
timestamps_path = Path(directory,'timeStamps.csv')
if timestamps_path.exists():
    timestamps = pd.read_csv(timestamps_path)
    frame_rate = round(1/(np.mean(np.diff(timestamps.iloc[:,1]))/1000))  # fps
    print(f'Acquisition with DAQ, frame rate = {frame_rate} fps')
else:
    frame_rate = 16  # fps /!\ CHANGE ACCORDING TO YOUR DATA
    print(f'Acquisition with Webcam, frame rate = {frame_rate} fps')
with open("//10.69.168.1/crnldata/forgetting/Carla/Cheeseboard/Reward_position.txt", "r") as file:
    text = file.read()  
numbers = re.findall(r"[-+]?\d*\.\d+|\d+", text)
reward_x, reward_y = map(float, numbers)

# Define parameters
pixel_to_cm = 5.05  
reward_zone = 8 * pixel_to_cm if "Training" in filename else 20 * pixel_to_cm  # 8 cm for training, 20 cm for test
table_center_x, table_center_y = 920, 637  # Center of the cheeseboard table on the video
table_radius = 606 / 2
min_stay_at_reward = 2 * frame_rate  # 5 seconds

# Define functions
def calculate_relative_distance(x1, y1, x2, y2):
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

def calculate_distance_run(x_coords, y_coords):
    distances = np.sqrt(np.diff(x_coords) ** 2 + np.diff(y_coords) ** 2)
    for i in range(1, len(distances) - 1):
        if np.isnan(distances[i]):
            neighbors = [distances[i-1], distances[i+1]]
            distances[i] = np.mean([x for x in neighbors if not np.isnan(x)])
    total_distance_cm = np.nansum(distances) / pixel_to_cm  # Convert to cm
    return total_distance_cm, distances

def remove_abnormal_speed(x, y, max_speed):
    distances = np.sqrt(np.diff(x) ** 2 + np.diff(y) ** 2)  
    for i in range(1, len(distances) - 1):
        if np.isnan(distances[i]):
            neighbors = [distances[i-1], distances[i+1]]
            distances[i] = np.mean([x for x in neighbors if not np.isnan(x)])
    mask = np.insert(distances <= max_speed, 0, True)  # Keep first point
    return x[mask], y[mask]

# Remove uncertain location predictions (likelihood < 0.9)
df.iloc[:, 0] = df.apply(lambda row: row.iloc[0] if row.iloc[2] > 0.9 else np.nan, axis=1)
df.iloc[:, 1] = df.apply(lambda row: row.iloc[1] if row.iloc[2] > 0.9 else np.nan, axis=1)

X = df.iloc[:, 0]
Y = df.iloc[:, 1]

# Separate the individual's positions into x and y coordinates
individual_x = np.array(X.values)
individual_y = np.array(Y.values)

# Define when the mouse is on the cheeseboard (start)
for i, x in enumerate(individual_x):
    y = individual_y[i]
    if calculate_relative_distance(x, y, table_center_x, table_center_y) >= table_radius:
        individual_x[i] = np.nan
        individual_y[i] = np.nan

x_start = individual_x[~np.isnan(individual_x)][0] if np.any(~np.isnan(individual_x)) else None # first non nan value
y_start = individual_y[~np.isnan(individual_y)][0] if np.any(~np.isnan(individual_y)) else None # first non nan value

start_frame = np.where(individual_x == x_start)[0].item()
for i in range(len(individual_x)-1, 0, -1): # Find the last non-NaN value which is not isolated
    if not np.isnan(individual_x[i]) and not np.isnan(individual_x[i-1]):
        last_frame = i
        break
    
if timestamps_path.exists():
    start_time = timestamps.iloc[start_frame,1].item() / 1000
    end_time = timestamps.iloc[-1,1].item() / 1000
    duration_trial = end_time - start_time
else:
    duration_trial = (last_frame - start_frame) / frame_rate
print(f'Total trial duration: {round(duration_trial)} sec')

# Define when the mouse eats the reward (end)
found_reward_frame = np.nan
consecutive_count = 0
for i, (x, y) in enumerate(zip(individual_x, individual_y)):
    if calculate_relative_distance(x, y, reward_x, reward_y) <= reward_zone:
        consecutive_count += 1
    else:
        consecutive_count = 0  
    if consecutive_count > min_stay_at_reward:
        found_reward_frame = (i - min_stay_at_reward)
        break
    
if timestamps_path.exists():
    found_reward_time = timestamps.iloc[found_reward_frame,1].item() / 1000
    latency = (found_reward_time - start_time)
else:
    latency = (found_reward_frame - start_frame) / frame_rate
print(f'Latency to reward: {round(latency)} sec')

# Define the time spent inside the reward zone
time_spent_in_zone = 0

for i, (x, y) in enumerate(zip(individual_x, individual_y)):
    if calculate_relative_distance(x, y, reward_x, reward_y) <= reward_zone:
        time_spent_in_zone += 1

time_spent_in_zone = time_spent_in_zone / frame_rate
print(f'Time spent in reward zone: {round(time_spent_in_zone)} sec ({round(time_spent_in_zone / duration_trial * 100)} %)')

# Define the number of times the mouse enters the reward zone
enter_reward_zone = 0
consecutive_count = 0
for i, (x, y) in enumerate(zip(individual_x, individual_y)):
    if calculate_relative_distance(x, y, reward_x, reward_y) <= reward_zone:
        if consecutive_count == 0:
            enter_reward_zone += 1
            consecutive_count = 1
    else:
        consecutive_count = 0   

print(f'Crossings in the reward zone: {enter_reward_zone}')

total_distance, distances = calculate_distance_run(individual_x[start_frame:last_frame], individual_y[start_frame:last_frame])
print(f"Total distance run: {round(total_distance)} cm")

distance_to_reward, distances_to_reward = calculate_distance_run(individual_x[start_frame:found_reward_frame], individual_y[start_frame:found_reward_frame])
print(f'Distance run to reward: {round(distance_to_reward)} cm')

print(f"Crossings per meter: {round(enter_reward_zone / (round(distance_to_reward) / 100), 2)}")

print(f"Average speed: {round(np.nanmean(distances_to_reward)/pixel_to_cm*frame_rate,2)} cm/s")

# Create the plot
fig, ax = plt.subplots(figsize=(5, 5)) 

# Remove NaN values to get a cleaner plot
individual_x_filt = individual_x[~np.isnan(individual_x)]        
individual_y_filt = individual_y[~np.isnan(individual_y)]

# Plot individual positions over time
cmap = plt.get_cmap('gnuplot2')
norm = plt.Normalize(vmin=0, vmax=len(individual_x_filt))

for i in range(1, len(individual_x_filt)):
    ax.plot(individual_x_filt[i-1:i+1], individual_y_filt[i-1:i+1], color=cmap(norm(i)), linewidth=3)
#plt.plot(individual_x_filt, individual_y_filt, label="Individual's Path", color='b')

plt.scatter(x_start, y_start, color='black', s=100, label='Start')
plt.scatter(reward_x, reward_y, color='red', s=100, label='Reward')

# Draw reward zone circle
reward_circle = plt.Circle((reward_x, reward_y), reward_zone, color='r', fill=False)
plt.gca().add_patch(reward_circle)  

# Draw cheeseboard circle
table_circle = plt.Circle((table_center_x, table_center_y), table_radius, color='k', fill=False)
plt.gca().add_patch(table_circle) 

# Add labels and title
ax.set_aspect('equal')
ax.invert_yaxis()
plt.title(f'Mouse Path On Cheeseboard Maze')
plt.xlabel('X Position')
plt.ylabel('Y Position')
plt.legend(loc='upper left')
plt.show()

Acquisition with Webcam, frame rate = 16 fps
Total trial duration: 79 sec


ValueError: cannot convert float NaN to integer

In [ ]:

X = df.iloc[:, 0]
Y = df.iloc[:, 1]

# Separate the individual's positions into x and y coordinates
individual_x = np.array(X.values)
individual_y = np.array(Y.values)


# Define when the mouse is on the cheeseboard (start)
for i, x in enumerate(individual_x):
    y = individual_y[i]
    if calculate_relative_distance(x, y, table_center_x, table_center_y) >= table_radius:
        individual_x[i] = np.nan
        individual_y[i] = np.nan


In [35]:
table_radius

135.0

In [34]:
individual_x

array([      nan,       nan,       nan, ..., 956.1788 , 956.0767 ,
       956.17035], dtype=float32)